# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [274]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [275]:
#pip install opencv-python

In [276]:
import os
import imageio
import datetime
import os
import math
import cv2 as cv

We set the random seed so that the results don't vary drastically.

In [277]:
import numpy as np
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [278]:
train_doc = np.random.permutation(open('D:/030_Neural Networks Project - Gesture Recognition/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('D:/030_Neural Networks Project - Gesture Recognition/Project_data/val.csv').readlines())
batch_size = 20

In [279]:
train_doc

array(['WIN_20180926_16_54_08_Pro_Right_Swipe_new;Right_Swipe_new;1\n',
       'WIN_20180925_18_02_58_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180925_17_33_08_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180925_17_51_17_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_17_35_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180926_17_30_47_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_23_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180926_17_12_26_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180926_17_29_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_32_11_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180926_17_06_01_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180907_16_52_18_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180907_16_39_19_Pro_Left Swipe_new_Left Swipe_new;Left Swipe_new_Left Swipe_new;0\n',
       'WIN_20180925_17_19_16_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n',
       'WIN_20180907_1

In [280]:
val_doc

array(['WIN_20180926_17_24_20_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180907_15_54_30_Pro_Thumbs Up_new;Thumbs Up_new;4\n',
       'WIN_20180926_17_08_11_Pro_Stop_new;Stop_new;2\n',
       'WIN_20180925_18_01_40_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180907_16_16_48_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180907_16_33_15_Pro_Thumbs Down_new;Thumbs Down_new;3\n',
       'WIN_20180926_16_46_22_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_20180907_16_25_44_Pro_Stop Gesture_new;Stop Gesture_new;2\n',
       'WIN_20180926_17_05_38_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180907_15_55_06_Pro_Right Swipe_new;Right Swipe_new;1\n',
       'WIN_20180926_16_57_50_Pro_Thumbs_Down_new;Thumbs_Down_new;3\n',
       'WIN_20180926_17_24_12_Pro_Right_Swipe_new;Right_Swipe_new;1\n',
       'WIN_20180907_15_57_43_Pro_Right Swipe_new;Right Swipe_new;1\n',
       'WIN_20180926_17_05_38_Pro_Left_Swipe_new;Left_Swipe_new;0\n',
       'WIN_

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [281]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,30)]#create a list of image numbers you want to use for a particular video
    x = len(img_idx) #x is the number of images you use for each video
    y = 128 #(y,z) is the final size of the input images and 3 is the number of channels RGB
    z = 128 #(y,z) is the final size of the input images and 3 is the  number of channels RGB
    while True:
        t = np.random.permutation(folder_list)
        num_batches =  math.floor(len(t)/batch_size)# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    if image.shape[0] != image.shape[1]:
                        image=image[0:120,20:140]
                        
                    # Resizing the image
                    image = cv.resize(image, (y, z), interpolation=cv.INTER_AREA)
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - np.percentile(image[:,:,0],5) / (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5)) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - np.percentile(image[:,:,1],5) / (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,0],5))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - np.percentile(image[:,:,2],5) / (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,0],5))#normalise and feed in the image
                    

                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

    
        # write the code for the remaining data points which are left after full batches
        #Code for remaining values in batch
         # write the code for the remaining data points which are left after full batches
        #Code for remaining values in batch
        total_folder_processed = num_batches * batch_size # Finding total images procees
        remaining_batch_size = len(folder_list) -  total_folder_processed # finding remaining images to process
        batch_data = np.zeros((remaining_batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((remaining_batch_size,5)) # batch_labels is the one hot representation of the output
            
        for folder in range(remaining_batch_size): # iterate over remaining images
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*remaining_batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                image = imageio.imread(source_path+'/'+ t[folder + (batch*remaining_batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes
                # Cropping non symmetric frames
                if image.shape[0] != image.shape[1]:
                    image=image[0:120,20:140]

                # Resizing the image
                image = cv.resize(image, (y, z), interpolation=cv.INTER_AREA)

                batch_data[folder,idx,:,:,0] = image[:,:,0] - np.percentile(image[:,:,0],5) / (np.percentile(image[:,:,0],95) - np.percentile(image[:,:,0],5)) #normalise and feed in the image
                batch_data[folder,idx,:,:,1] = image[:,:,1] - np.percentile(image[:,:,1],5) / (np.percentile(image[:,:,1],95) - np.percentile(image[:,:,0],5)) #normalise and feed in the image
                batch_data[folder,idx,:,:,2] = image[:,:,2] - np.percentile(image[:,:,2],5) / (np.percentile(image[:,:,2],95) - np.percentile(image[:,:,0],5)) #normalise and feed in the image

            batch_labels[folder, int(t[folder + (batch*remaining_batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [282]:
curr_dt_time = datetime.datetime.now()
train_path = 'D:/030_Neural Networks Project - Gesture Recognition/Project_data/train'
val_path = 'D:/030_Neural Networks Project - Gesture Recognition/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [283]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

import matplotlib.pyplot as plt

from keras.optimizers import Adam
optimiser = Adam(0.001) #write your optimizer

In [284]:
input_shape = (len([i for i in range(0,30)]),128,128,3)
no_classes = 5

### CNN_3D

#### model_cnn_3d_1

In [285]:
model_cnn_3d_1 = Sequential()
#Layer 1
model_cnn_3d_1.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model_cnn_3d_1.add(BatchNormalization())
model_cnn_3d_1.add(Activation('relu'))
model_cnn_3d_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Layer 2
model_cnn_3d_1.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model_cnn_3d_1.add(BatchNormalization())
model_cnn_3d_1.add(Activation('relu'))
model_cnn_3d_1.add(MaxPooling3D(pool_size=(1,2,2)))

#Flatten Layers
model_cnn_3d_1.add(Flatten())

model_cnn_3d_1.add(Dense(256, activation='relu'))

model_cnn_3d_1.add(Dense(128, activation='relu'))
#softmax layer
model_cnn_3d_1.add(Dense(no_classes, activation='softmax'))

model_cnn_3d_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_cnn_3d_1.summary())

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_76 (Conv3D)          (None, 30, 128, 128, 8)   656       
                                                                 
 batch_normalization_61 (Bat  (None, 30, 128, 128, 8)  32        
 chNormalization)                                                
                                                                 
 activation_76 (Activation)  (None, 30, 128, 128, 8)   0         
                                                                 
 max_pooling3d_74 (MaxPoolin  (None, 15, 64, 64, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_77 (Conv3D)          (None, 15, 64, 64, 16)    3472      
                                                                 
 batch_normalization_62 (Bat  (None, 15, 64, 64, 16) 

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_cnn_3d_1_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_cnn_3d_1_-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_cnn_3d_1_history = model_cnn_3d_1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  D:/030_Neural Networks Project - Gesture Recognition/Project_data/train ; batch size = 20
Epoch 1/10
 7/34 [=====>........................] - ETA: 4:34 - loss: 117.7166 - categorical_accuracy: 0.1643

In [ ]:
 # summarize history for accuracy
plt.plot(model_cnn_3d_1_history.history['categorical_accuracy']) # Get 
plt.plot(model_cnn_3d_1_history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
 # summarize history for loss
plt.plot(model_cnn_3d_1_history.history['loss'])
plt.plot(model_cnn_3d_1_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#### model_cnn_3d_2

In [ ]:
model_cnn_3d_2 = Sequential()
#Layer 1
model_cnn_3d_2.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model_cnn_3d_2.add(BatchNormalization())
model_cnn_3d_2.add(Activation('relu'))
#model_cnn_3d_2.add(Dropout(0.25))
model_cnn_3d_2.add(MaxPooling3D(pool_size=(2,2,2)))

#Layer 2
model_cnn_3d_2.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model_cnn_3d_2.add(BatchNormalization())
model_cnn_3d_2.add(Activation('relu'))
#model_cnn_3d_2.add(Dropout(0.25))
model_cnn_3d_2.add(MaxPooling3D(pool_size=(2,2,2)))

model_cnn_3d_2.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
model_cnn_3d_2.add(BatchNormalization())
model_cnn_3d_2.add(Activation('relu'))
#model_cnn_3d_2.add(Dropout(0.25))
model_cnn_3d_2.add(MaxPooling3D(pool_size=(2,2,2)))

model_cnn_3d_2.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
model_cnn_3d_2.add(Activation('relu'))
model_cnn_3d_2.add(Dropout(0.25))

model_cnn_3d_2.add(MaxPooling3D(pool_size=(1,2,2)))

#Flatten Layers
model_cnn_3d_2.add(Flatten())

model_cnn_3d_2.add(Dense(256, activation='relu'))
#model.add(Dropout(0.5))

model_cnn_3d_2.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))

#softmax layer
model_cnn_3d_2.add(Dense(no_classes, activation='softmax'))


model_cnn_3d_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_cnn_3d_2.summary())

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_cnn_3d_2_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_cnn_3d_2-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_cnn_3d_2_history = model_cnn_3d_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
 # summarize history for accuracy
plt.plot(model_cnn_3d_2_history.history['categorical_accuracy']) # Get 
plt.plot(model_cnn_3d_2_history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
 # summarize history for accuracy
plt.plot(model_cnn_3d_2_history.history['categorical_accuracy']) # Get 
plt.plot(model_cnn_3d_2_history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

#### model_cnn_3d_3

In [249]:
model_cnn_3d_3 = Sequential()
#Layer 1
model_cnn_3d_3.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
model_cnn_3d_3.add(BatchNormalization())
model_cnn_3d_3.add(Activation('relu'))
model_cnn_3d_3.add(Dropout(0.25))
model_cnn_3d_3.add(MaxPooling3D(pool_size=(2,2,2)))

#Layer 2
model_cnn_3d_3.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
model_cnn_3d_3.add(BatchNormalization())
model_cnn_3d_3.add(Activation('relu'))
model_cnn_3d_3.add(Dropout(0.25))
model_cnn_3d_3.add(MaxPooling3D(pool_size=(2,2,2)))

model_cnn_3d_3.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
model_cnn_3d_3.add(BatchNormalization())
model_cnn_3d_3.add(Activation('relu'))
model_cnn_3d_3.add(Dropout(0.25))
model_cnn_3d_3.add(MaxPooling3D(pool_size=(2,2,2)))

model_cnn_3d_3.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
model_cnn_3d_3.add(Activation('relu'))
model_cnn_3d_3.add(Dropout(0.25))

model_cnn_3d_3.add(MaxPooling3D(pool_size=(1,2,2)))

#Flatten Layers
model_cnn_3d_3.add(Flatten())

model_cnn_3d_3.add(Dense(256, activation='relu'))
model_cnn_3d_3.add(Dropout(0.5))

model_cnn_3d_3.add(Dense(128, activation='relu'))
model_cnn_3d_3.add(Dropout(0.5))

#softmax layer
model_cnn_3d_3.add(Dense(no_classes, activation='softmax'))


model_cnn_3d_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_cnn_3d_3.summary())

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model_name = 'model_cnn_3d_3_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_cnn_3d_3-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

model_cnn_3d_3_history = model_cnn_3d_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
 # summarize history for accuracy
plt.plot(model_cnn_3d_3_history.history['categorical_accuracy']) # Get 
plt.plot(model_cnn_3d_3_history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

In [ ]:
 # summarize history for accuracy
plt.plot(model_cnn_3d_3_history.history['categorical_accuracy']) # Get 
plt.plot(model_cnn_3d_3_history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

### CNN_LSTM

In [ ]:
#STILL ON THE WORKS

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [250]:
from keras.optimizers import Adam
optimiser = Adam(0.001) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_62 (Conv3D)          (None, 30, 128, 128, 8)   656       
                                                                 
 batch_normalization_47 (Bat  (None, 30, 128, 128, 8)  32        
 chNormalization)                                                
                                                                 
 activation_62 (Activation)  (None, 30, 128, 128, 8)   0         
                                                                 
 max_pooling3d_60 (MaxPoolin  (None, 15, 64, 64, 8)    0         
 g3D)                                                            
                                                                 
 conv3d_63 (Conv3D)          (None, 15, 64, 64, 16)    3472      
                                                                 
 batch_normalization_48 (Bat  (None, 15, 64, 64, 16) 

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [251]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [252]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=5, cooldown=4, verbose=1,mode='auto',epsilon=0.0001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [253]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [254]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

C:\Users\SERGIO~1\AppData\Local\Temp/ipykernel_17580/3383535347.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  D:/030_Neural Networks Project - Gesture Recognition/Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - ETA: 0s - loss: 84.8912 - categorical_accuracy: 0.3861 Source path =  D:/030_Neural Networks Project - Gesture Recognition/Project_data/val ; batch size = 30

Epoch 1: saving model to model_init_2022-11-1516_21_36.249555\model-00001-84.89118-0.38612-112.57608-0.36000.h5
23/23 [==============================] - 345s 15s/step - loss: 84.8912 - categorical_accuracy: 0.3861 - val_loss: 112.5761 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 2/10
23/23 [==============================] - ETA: 0s - loss: 8.1774 - categorical_accuracy: 0.6501 
Epoch 2: saving model to model_init_2022-11-1516_21_36.249555\model-00002-8.17737-0.65008-75.48442-0.41000.h5
23/23 [==============================] - 353s 15s/step - loss: 8.1774 - categorical_accuracy: 0.6501 - val_loss: 75.4844 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 3/10
23/2

KeyboardInterrupt: 

In [225]:
import matplotlib.pyplot as plt

In [230]:
 # summarize history for accuracy
plt.plot(history.history['categorical_accuracy']) # Get 
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable

In [229]:
 # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

TypeError: 'History' object is not subscriptable

# CNN with LSTM

In [ ]:
input_shape = (len([i for i in range(0,30)]),128,128,3)
model_ccn_lstm_1 = Sequential()
# layer 1
# input, with 8 convolutions for 5 images (128, 128, 3) shapee
model_ccn_lstm_1.add(TimeDistributed(Conv2D(8, (3,3), padding='same', strides=(2,2), activation='relu'),input_shape = input_shape))
# layer 2
# input, with 16 convolutions for 5 images
model_ccn_lstm_1.add(TimeDistributed(Conv2D(16, (3,3), padding='same', strides=(2,2), activation='relu')))
model_ccn_lstm_1.add(TimeDistributed(MaxPooling2D((2,2), strides=(2,2))))
# layer 3
# input, with 32 convolutions for 5 images
model_ccn_lstm_1.add(TimeDistributed(Conv2D(32, (3,3), padding='same', strides=(2,2), activation='relu')))
model_ccn_lstm_1.add(TimeDistributed(MaxPooling2D((2,2), strides=(2,2))))

# layer 4
# input, with 64 convolutions for 5 images
#model_ccn_lstm .add(TimeDistributed(Conv2D(64, (3,3), padding='same', strides=(2,2), activation='relu')))
#model_ccn_lstm .add(TimeDistributed(MaxPooling2D((2,2), strides=(2,2))))


model_ccn_lstm_1.add(TimeDistributed(BatchNormalization()))
#model.add(Dropout(0.5))

model_ccn_lstm_1.add(TimeDistributed(Flatten()))

model_ccn_lstm_1.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))

## using LSTM as the RNN model along with softmax as our last layer.
model_ccn_lstm_1.add(LSTM(128, return_sequences=False))
model_ccn_lstm_1.add(Dense(5, activation='softmax')) # using Softmax as last layer

